In [1]:
import cv2
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Flatten, Dropout, Dense
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

Using TensorFlow backend.


### 自带的预处理方法

In [2]:
def get_train_data1(imgDir, size=224):
    imgDir = shuffle(imgDir)
    train_x = np.zeros((len(imgDir), size, size, 3), dtype=np.float32)
    train_y = np.zeros(len(imgDir), dtype=np.uint8)
    for index, file in enumerate(imgDir):        
        img = image.load_img(file, target_size=(size, size))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        train_x[index] = preprocess_input(x)    
        if 'dog' in file:
            train_y[index] = 1
    return train_x, train_y

In [3]:
# cat 0  dog 1
print('Proprecessing begin !')
t=time.time()
train_dir = glob.glob('train_all/*.jpg')
train_x, train_y = get_train_data1(train_dir)
t2 = time.time()
print(round(t2 - t, 2), 'seconds to preprocessing !')

Proprecessing begin !
1215.41 seconds to preprocessing !


## ------------------------------------------------------------------------------------------------------
### 50% Dropout

In [6]:
from keras.optimizers import SGD
from keras.applications.resnet50 import ResNet50
opt = SGD(lr=0.0001, momentum=0.9)
myinput = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', input_tensor=myinput, include_top=False)
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
# this is the model we will train
model = Model(myinput, predictions)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
best_model = ModelCheckpoint('model_best.h5', verbose=1, save_best_only=True)
model.fit(train_x, train_y, validation_split=0.2, shuffle=True, batch_size=16, epochs=10, callbacks=[best_model])
model.save('model_10.h5')

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
19990/20000 [============================>.] - ETA: 2s - loss: 0.1564 - acc: 0.9376
Epoch 00001: val_loss improved from inf to 0.04456, saving model to model_best.h5
20000/20000 [==============================] - 5523s 276ms/step - loss: 0.1564 - acc: 0.9375 - val_loss: 0.0446 - val_acc: 0.9854
Epoch 2/10
19990/20000 [============================>.] - ETA: 2s - loss: 0.0777 - acc: 0.9699
Epoch 00002: val_loss improved from 0.04456 to 0.03630, saving model to model_best.h5
20000/20000 [==============================] - 5346s 267ms/step - loss: 0.0777 - acc: 0.9699 - val_loss: 0.0363 - val_acc: 0.9872
Epoch 3/10
19990/20000 [============================>.] - ETA: 2s - loss: 0.0541 - acc: 0.9809
Epoch 00003: val_loss improved from 0.03630 to 0.03114, saving model to model_best.h5
20000/20000 [==============================] - 5357s 268ms/step - loss: 0.0541 - acc: 0.9809 - val_loss: 0.0311 - val_acc: 0.9892
Epoch 4/10
19990/20000

## ------------------------------------------------------------------------------------------------------
### 75% Dropout

In [4]:
from keras.optimizers import SGD
from keras.applications.resnet50 import ResNet50
opt = SGD(lr=0.0001, momentum=0.9)
myinput = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', input_tensor=myinput, include_top=False)
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.75)(x)
predictions = Dense(1, activation='sigmoid')(x)
# this is the model we will train
model = Model(myinput, predictions)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
best_model = ModelCheckpoint('model_best.h5', verbose=1, save_best_only=True)
model.fit(train_x, train_y, validation_split=0.2, shuffle=True, batch_size=16, epochs=10, callbacks=[best_model])
model.save('model_10.h5')

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
19984/20000 [============================>.] - ETA: 3s - loss: 0.2106 - acc: 0.9092
Epoch 00001: val_loss improved from inf to 0.05018, saving model to model_best.h5
20000/20000 [==============================] - 5164s 258ms/step - loss: 0.2105 - acc: 0.9092 - val_loss: 0.0502 - val_acc: 0.9854
Epoch 2/10
19984/20000 [============================>.] - ETA: 3s - loss: 0.0819 - acc: 0.9699
Epoch 00002: val_loss improved from 0.05018 to 0.03850, saving model to model_best.h5
20000/20000 [==============================] - 5244s 262ms/step - loss: 0.0819 - acc: 0.9699 - val_loss: 0.0385 - val_acc: 0.9872
Epoch 3/10
19984/20000 [============================>.] - ETA: 3s - loss: 0.0610 - acc: 0.9772
Epoch 00003: val_loss improved from 0.03850 to 0.03489, saving model to model_best.h5
20000/20000 [==============================] - 5178s 259ms/step - loss: 0.0610 - acc: 0.9771 - val_loss: 0.0349 - val_acc: 0.9880
Epoch 4/10
19984/20000

## ------------------------------------------------------------------------------------------------------
### Clean data

In [5]:
import os
size = 224
model_test = load_model('model_best.h5')
cat_dir = glob.glob('cat/*.jpg')
for file_name in cat_dir:        
    img = image.load_img(file_name, target_size=(size, size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    if model_test.predict(x) > 0.5:
        print(file_name)
        os.remove(file_name)

cat\cat.10029.jpg
cat\cat.10266.jpg
cat\cat.10610.jpg
cat\cat.10712.jpg
cat\cat.11184.jpg
cat\cat.11222.jpg
cat\cat.11281.jpg
cat\cat.11565.jpg
cat\cat.12272.jpg
cat\cat.12499.jpg
cat\cat.1361.jpg
cat\cat.1962.jpg
cat\cat.2337.jpg
cat\cat.3202.jpg
cat\cat.3658.jpg
cat\cat.4085.jpg
cat\cat.4308.jpg
cat\cat.4360.jpg
cat\cat.4688.jpg
cat\cat.4986.jpg
cat\cat.5355.jpg
cat\cat.5418.jpg
cat\cat.5583.jpg
cat\cat.5795.jpg
cat\cat.5834.jpg
cat\cat.6304.jpg
cat\cat.6402.jpg
cat\cat.6655.jpg
cat\cat.7564.jpg
cat\cat.7655.jpg
cat\cat.7671.jpg
cat\cat.7703.jpg
cat\cat.7920.jpg
cat\cat.7968.jpg
cat\cat.8138.jpg
cat\cat.8456.jpg
cat\cat.8504.jpg
cat\cat.8828.jpg
cat\cat.9290.jpg
cat\cat.9596.jpg
cat\cat.9897.jpg


In [6]:
import os
size = 224
model_test = load_model('model_best.h5')
dog_dir = glob.glob('dog/*.jpg')
for file_name in dog_dir:        
    img = image.load_img(file_name, target_size=(size, size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    if model_test.predict(x) < 0.5:
        print(file_name)
        os.remove(file_name)

dog\dog.10225.jpg
dog\dog.10524.jpg
dog\dog.10801.jpg
dog\dog.10939.jpg
dog\dog.11299.jpg
dog\dog.11300.jpg
dog\dog.11526.jpg
dog\dog.11731.jpg
dog\dog.12223.jpg
dog\dog.2614.jpg
dog\dog.3074.jpg
dog\dog.3341.jpg
dog\dog.3920.jpg
dog\dog.4334.jpg
dog\dog.4690.jpg
dog\dog.5251.jpg
dog\dog.5529.jpg
dog\dog.5767.jpg
dog\dog.6256.jpg
dog\dog.6921.jpg
dog\dog.7.jpg
dog\dog.7076.jpg
dog\dog.7332.jpg
dog\dog.7413.jpg
dog\dog.7692.jpg
dog\dog.8671.jpg
dog\dog.9517.jpg


## ------------------------------------------------------------------------------------------------------
### 75% Dropout Retrain

In [3]:
# cat 0  dog 1
print('Proprecessing begin !')
t=time.time()
train_dir = glob.glob('clean_train/*.jpg')
train_x, train_y = get_train_data1(train_dir)
t2 = time.time()
print(round(t2 - t, 2), 'seconds to preprocessing !')

Proprecessing begin !
1170.75 seconds to preprocessing !


In [4]:
from keras.optimizers import SGD
from keras.applications.resnet50 import ResNet50
opt = SGD(lr=0.0001, momentum=0.9)
myinput = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', input_tensor=myinput, include_top=False)
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.75)(x)
predictions = Dense(1, activation='sigmoid')(x)
# this is the model we will train
model = Model(myinput, predictions)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
best_model = ModelCheckpoint('model_best.h5', verbose=1, save_best_only=True)
model.fit(train_x, train_y, validation_split=0.2, shuffle=True, batch_size=16, epochs=10, callbacks=[best_model])
model.save('model_10.h5')

Train on 19945 samples, validate on 4987 samples
Epoch 1/10
19936/19945 [============================>.] - ETA: 2s - loss: 0.1761 - acc: 0.9323
Epoch 00001: val_loss improved from inf to 0.04538, saving model to model_best.h5
19945/19945 [==============================] - 5182s 260ms/step - loss: 0.1762 - acc: 0.9323 - val_loss: 0.0454 - val_acc: 0.9880
Epoch 2/10
19936/19945 [============================>.] - ETA: 2s - loss: 0.0691 - acc: 0.9755
Epoch 00002: val_loss improved from 0.04538 to 0.03252, saving model to model_best.h5
19945/19945 [==============================] - 5002s 251ms/step - loss: 0.0691 - acc: 0.9755 - val_loss: 0.0325 - val_acc: 0.9908
Epoch 3/10
19936/19945 [============================>.] - ETA: 2s - loss: 0.0515 - acc: 0.9810
Epoch 00003: val_loss improved from 0.03252 to 0.02635, saving model to model_best.h5
19945/19945 [==============================] - 5040s 253ms/step - loss: 0.0515 - acc: 0.9810 - val_loss: 0.0264 - val_acc: 0.9916
Epoch 4/10
19936/19945

## ------------------------------------------------------------------------------------------------------
### Drawing model

In [4]:
from keras.models import load_model
model = load_model('model_best.h5')
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model.png',show_shapes=True)

## ------------------------------------------------------------------------------------------------------
### Testing

In [2]:
import os
size = 224
test_dir = glob.glob('test1/*.jpg')
mytest = np.zeros((12500, size, size, 3), dtype=np.float32)
test_path = r'E:\final\test1'
for file_name in (test_dir):
    index = int(file_name[6:-4]) - 1
    img = image.load_img(file_name, target_size=(size, size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    mytest[index] = preprocess_input(x)

## ------------------------------------------------------------------------------------------------------
### Write in file

In [3]:
##predict
with open('file3.csv','w') as f:
    f.write('id,label\n')

model_test = load_model('model_best.h5')

with open('file3.csv','a') as f:
    for i in range(len(test_dir)):
        predict = model_test.predict(mytest[i:i+1])
        predict = predict[0][0]
        f.write('{},{}\n'.format(i+1,predict))